In [242]:
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [243]:
#Plotly with jupyter offline : 
init_notebook_mode(connected=True)

In [244]:
df = pd.read_csv('map.csv', index_col=0, names = ['nb_attack', 'target', 'attacker'], header = 0)

In [245]:
df.head()

,nb_attack,target,attacker
0,92157,united states,united states
1,26877,united arab emirates,china
2,13059,united states,china
3,6424,united states,ukraine
4,5202,united arab emirates,netherlands


In [246]:
df_country = pd.DataFrame(list(set(list(df['attacker'].unique()) + list(df['target'].unique()))), columns = ['country'] )

In [247]:
geolocator = Nominatim()
df_country['lattitude_longitude'] = df_country['country'].apply(lambda x : geolocator.geocode(x,  timeout=10)[1])

In [248]:
# Geolocator is wrong for united arabe emirates 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (25.61552, 55.84951) if x['country'] == 'united arab emirates' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (39.904211, 116.407395) if x['country'] == 'china' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (37.566535, 126.9779692) if x['country'] == 'south korea' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (51.3030, 0.0732) if x['country'] == 'united kingdom' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (40.18658704808404, 44.78151767883003) if x['country'] == 'armenia' else x['lattitude_longitude'], axis = 'columns') 

In [249]:
df_country['latitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[0])
df_country['longitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[1])
df_country['attacker'] = df_country['country']
df_country = df_country.drop(['lattitude_longitude', 'country'], axis = 'columns')

In [250]:
df = pd.merge(df, df_country, how = 'left', on = ['attacker'])

In [251]:
df_country['latitude_target'] = df_country['latitude_attacker']
df_country['longitude_target'] = df_country['longitude_attacker']
df_country['target'] = df_country['attacker']
df_country = df_country.drop(['longitude_attacker', 'attacker', 'latitude_attacker'], axis = 'columns')

In [252]:
df = pd.merge(df, df_country, how = 'left', on = ['target'])

In [253]:
df.head()

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target
0,92157,united states,united states,39.783730,-100.445882,39.78373,-100.445882
1,26877,united arab emirates,china,39.904211,116.407395,25.61552,55.849510
2,13059,united states,china,39.904211,116.407395,39.78373,-100.445882
3,6424,united states,ukraine,49.487197,31.271832,39.78373,-100.445882
4,5202,united arab emirates,netherlands,52.237989,5.534607,25.61552,55.849510


- Map monde, sympa pour l'oral, remplacer 'equirectangular' par 'orthographic'

In [254]:
df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

airports = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_airports['long'],
        lat = df_airports['lat'],
        hoverinfo = 'text',
        text = df_airports['airport'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]
        
flight_paths = []
for i in range( len( df_flight_paths ) ):
    flight_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [ df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i] ],
            lat = [ df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
            opacity = float(df_flight_paths['cnt'][i])/float(df_flight_paths['cnt'].max()),
        )
    )
    
layout = dict(
        showlegend = False, 
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),#dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=flight_paths + airports, layout=layout )
iplot( fig, filename='d3-world-map')

In [255]:
df['lat'] = df.apply(lambda x :[x['latitude_attacker'], x['latitude_target']], axis ='columns')
df['lon'] =  df.apply(lambda x :[x['longitude_attacker'], x['longitude_target']], axis ='columns')

In [256]:
df['line'] = ''
#df['line'] = df['line'].apply(lambda x : {'color' : 'red', 'width' : 1}) 
df['line'] = df.apply(lambda x : {'color' : 'red', 'width' : (x['nb_attack'] / sum(df['nb_attack']))*80}, axis='columns')

In [257]:
df['opacity'] = 0
df['opacity'] = df['opacity'].apply(lambda x : 0.48577680525164113)

In [258]:
df['type'] = ''
df['type'] = df['type'].apply(lambda x : 'scattergeo')

In [259]:
df['mode'] = ''
df['mode'] = df['mode'].apply(lambda x : 'lines')

In [260]:
df['text'] = df.apply(lambda x : '{} - {} : {} times'.format(x['attacker'],x['target'], x['nb_attack']), axis='columns')

In [261]:
df

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target,lat,lon,line,opacity,type,mode,text
0,92157,united states,united states,39.783730,-100.445882,39.783730,-100.445882,"[39.7837304, 39.7837304]","[-100.4458825, -100.4458825]","{'color': 'red', 'width': 34.083306365863805}",0.485777,scattergeo,lines,united states - united states : 92157 times
1,26877,united arab emirates,china,39.904211,116.407395,25.615520,55.849510,"[39.904211, 25.61552]","[116.407395, 55.84951]","{'color': 'red', 'width': 9.940178447598354}",0.485777,scattergeo,lines,china - united arab emirates : 26877 times
2,13059,united states,china,39.904211,116.407395,39.783730,-100.445882,"[39.904211, 39.7837304]","[116.407395, -100.4458825]","{'color': 'red', 'width': 4.829735102399334}",0.485777,scattergeo,lines,china - united states : 13059 times
3,6424,united states,ukraine,49.487197,31.271832,39.783730,-100.445882,"[49.4871968, 39.7837304]","[31.2718321, -100.4458825]","{'color': 'red', 'width': 2.375849475290093}",0.485777,scattergeo,lines,ukraine - united states : 6424 times
4,5202,united arab emirates,netherlands,52.237989,5.534607,25.615520,55.849510,"[52.2379891, 25.61552]","[5.53460738161551, 55.84951]","{'color': 'red', 'width': 1.9239055059867782}",0.485777,scattergeo,lines,netherlands - united arab emirates : 5202 times
5,4761,spain,china,39.904211,116.407395,40.002803,-4.003104,"[39.904211, 40.0028028]","[116.407395, -4.003104]","{'color': 'red', 'width': 1.7608062502889372}",0.485777,scattergeo,lines,china - spain : 4761 times
6,2754,united arab emirates,united states,39.783730,-100.445882,25.615520,55.849510,"[39.7837304, 25.61552]","[-100.4458825, 55.84951]","{'color': 'red', 'width': 1.0185382090518238}",0.485777,scattergeo,lines,united states - united arab emirates : 2754 times
7,2523,united states,india,22.351115,78.667743,39.783730,-100.445882,"[22.3511148, 39.7837304]","[78.6677428, -100.4458825]","{'color': 'red', 'width': 0.9331052655910499}",0.485777,scattergeo,lines,india - united states : 2523 times
8,2502,united states,south korea,37.566535,126.977969,39.783730,-100.445882,"[37.566535, 39.7837304]","[126.9779692, -100.4458825]","{'color': 'red', 'width': 0.9253386343673432}",0.485777,scattergeo,lines,south korea - united states : 2502 times
9,2458,singapore,china,39.904211,116.407395,1.290475,103.852036,"[39.904211, 1.2904753]","[116.407395, 103.8520359]","{'color': 'red', 'width': 0.9090656927557672}",0.485777,scattergeo,lines,china - singapore : 2458 times


In [262]:
df['showlegend'] = False
data_norse = df[['lat', 'line', 'lon', 'mode', 'opacity','type', 'showlegend']].to_dict(orient='records')
df_intra_pays = df[df['attacker'] == df['target']]

labels = [ dict(
        type = 'scattergeo',
        hoverinfo = 'text',
        lon = df['longitude_attacker'],
        lat = df['latitude_attacker'],
        text = df['attacker'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=1,
                color='rgba(68, 68, 68, 0)'
            )   
        ),
        legendgroup = 'group1', # this can be any string, not just "group"
        name = 'Flux inter pays',
        showlegend = True
    
)]

cases = []
cases.append(go.Scattergeo(
    hoverinfo = 'text',
    lon = df_intra_pays['longitude_attacker'], 
    lat = df_intra_pays['latitude_attacker'],
    text = df_intra_pays['attacker'],
    marker = dict(
        size = df_intra_pays['nb_attack'].apply(lambda x :  x / sum(df['nb_attack']) * 80),
        color = 'rgb(33,113,181)',
        line = dict(width = 0)
    ),
    legendgroup = 'group2', # this can be any string, not just "group"
    name = 'Flux intra pays',
    mode = 'markers',
    showlegend = True
    
) )

 
layout = dict(
    title = 'Flux des cyberattaques mondiale en mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict(data = data_norse + cases + labels, layout=layout )
iplot(fig, filename='d3-world-map')

In [263]:
# 1ère cartes : Représentation des cyber attaques 
# 2 ème carte : cibles des cyber-attaques 
# 3 ème carte : origine des attaques 

In [317]:
df_code_country = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')[['CODE', 'COUNTRY']]
df_code_country['country'] = df_code_country['COUNTRY'].apply(lambda x : x.lower())
df_code_country = df_code_country.drop(['COUNTRY'], axis = 'columns')

In [318]:
sum_target = df.groupby(['target'])['nb_attack'].sum()
df_target = df.set_index(['target'])[['latitude_target', 'longitude_target']]
df_target['sum_target'] = sum_target
df_target = df_target.reset_index()
df_target = df_target.drop_duplicates('target')
df_target['country'] = df_target['target']
df_target = df_target.drop(['target'], axis = 'columns')

In [321]:
df_target = pd.merge(df_target, df_code_country, how = 'left', on = ['country'])

In [323]:
df_target.head()

,latitude_target,longitude_target,sum_target,country,CODE
0,39.783730,-100.445882,137702,united states,USA
1,25.615520,55.849510,45635,united arab emirates,ARE
2,40.002803,-4.003104,7740,spain,ESP
3,1.290475,103.852036,2702,singapore,SGP
4,64.573154,11.528036,1828,norway,NOR


In [324]:
sum_attacker = df.groupby(['attacker'])['nb_attack'].sum()
df_attacker = df.set_index(['attacker'])[['latitude_attacker', 'longitude_attacker']]
df_attacker['sum_attacker'] = sum_attacker
df_attacker = df_attacker.reset_index()
df_attacker = df_attacker.drop_duplicates('attacker')
df_attacker['country'] = df_attacker['attacker']
df_attacker = df_attacker.drop(['attacker'], axis = 'columns')

In [325]:
df_attacker = pd.merge(df_attacker, df_code_country, how = 'left', on = ['country'])

In [327]:
df_attacker.head()

,latitude_attacker,longitude_attacker,sum_attacker,country,CODE
0,39.783730,-100.445882,97988,united states,USA
1,39.904211,116.407395,52725,china,CHN
2,49.487197,31.271832,6680,ukraine,UKR
3,52.237989,5.534607,7854,netherlands,NLD
4,22.351115,78.667743,3183,india,IND


In [366]:
colorscale=[[0, 'rgb(166,206,227)'], [0.25, 'rgb(31,120,180)'], [0.45, 'rgb(178,223,138)'], [0.65, 'rgb(51,160,44)'], [0.85, 'rgb(251,154,153)'], [1, 'rgb(227,26,28)']]


data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_target['CODE'],
        z = np.log(df_target['sum_target']).astype(float),
        text = df_target['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Cibles des cyberattaques <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [367]:
colorscale=[[0, 'rgb(166,206,227)'], [0.25, 'rgb(31,120,180)'], [0.45, 'rgb(178,223,138)'], [0.65, 'rgb(51,160,44)'], [0.85, 'rgb(251,154,153)'], [1, 'rgb(227,26,28)']]

data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_attacker['CODE'],
        z = np.log(df_attacker['sum_attacker']).astype(float),
        text = df_attacker['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Origine des cyberattaques <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )